<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/embedding_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

from torchsummary import summary

class SiameseNetwork(nn.Module):
    def __init__(self, in_dimension=512, out_dimension=1024):
        super(SiameseNetwork, self).__init__()

        self.out_dimension = out_dimension

        self.cnn = nn.Conv2d(in_channels=in_dimension, out_channels=out_dimension, kernel_size=3, padding='same')

        self.linear_sub_1 = nn.LazyLinear(out_features=256)
        self.linear_sub_2 = nn.LazyLinear(out_features=1024)
        self.global_pooling = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward_once(self, x):
    # def forward(self, x):
        # Forward pass
        x = self.cnn(x)

        global_pooling = self.global_pooling(x)

        attention_score = torch.transpose(global_pooling, -3, -1)
        attention_score = self.linear_sub_1(attention_score)
        attention_score = self.relu(attention_score)
        attention_score = self.linear_sub_2(attention_score)
        attention_score = self.sigmoid(attention_score)
        attention_score = torch.transpose(attention_score, -3, -1)

        output = torch.mul(x, attention_score)

        output = torch.reshape(output, (-1, self.out_dimension))
        return output

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward_once(input1)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        return output1, output2


network = SiameseNetwork(1536, 1024).to(device='cuda')
summary(network, ((1536, 14, 14), (1536, 14, 14)))
tensor = torch.randn(4, 1536, 14, 14).to(device='cuda')
print(network(tensor, tensor).shape)